In [1]:
import pandas as pd
import numpy as np
import dask.dataframe as dd
import missingno as msno
import seaborn as sns
import matplotlib.pyplot as plt
from scipy import stats
from tqdm import tqdm_notebook
np.set_printoptions(suppress=True)
from sklearn import preprocessing
from tqdm import tqdm
import decimal
from itertools import combinations
from random import shuffle
from time import sleep
tqdm.pandas()
from sklearn.preprocessing import LabelEncoder
import gc
import datetime
from sklearn.model_selection import train_test_split, KFold, GroupKFold
import os
from sklearn.metrics import roc_auc_score
plt.style.use('ggplot')
np.set_printoptions(suppress=True)
import random
import lightgbm as lgb

In [2]:
#Always seed the randomness of this universe
def seed_everything(seed=51):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)

In [3]:
seed_everything(seed=51)

In [4]:
pd.set_option('display.max_rows', 2000)
pd.set_option('display.max_columns', 2000)

In [5]:
"""
train_dtypes = pd.read_csv('../input/train9_dtypes.csv')
test_dtypes = pd.read_csv('../input/test9_dtypes.csv')
train_dtypes.index = train_dtypes['TransactionID']
del train_dtypes['TransactionID']
test_dtypes.index = test_dtypes['TransactionID']
del test_dtypes['TransactionID']
train_dtypes = train_dtypes.iloc[:,0]
test_dtypes = test_dtypes.iloc[:,0]
print(type(train_dtypes),type(test_dtypes))
train_dtypes = train_dtypes.to_dict()
test_dtypes = test_dtypes.to_dict()
gc.collect()
"""

"\ntrain_dtypes = pd.read_csv('../input/train9_dtypes.csv')\ntest_dtypes = pd.read_csv('../input/test9_dtypes.csv')\ntrain_dtypes.index = train_dtypes['TransactionID']\ndel train_dtypes['TransactionID']\ntest_dtypes.index = test_dtypes['TransactionID']\ndel test_dtypes['TransactionID']\ntrain_dtypes = train_dtypes.iloc[:,0]\ntest_dtypes = test_dtypes.iloc[:,0]\nprint(type(train_dtypes),type(test_dtypes))\ntrain_dtypes = train_dtypes.to_dict()\ntest_dtypes = test_dtypes.to_dict()\ngc.collect()\n"

In [6]:
%%time
#train_identity
train_identity = dd.read_csv('../input/train_identity.csv')
train_identity = train_identity.compute()
train_identity = train_identity.reset_index(drop=True)
print("train_identity finished")
#test_identity
test_identity = dd.read_csv('../input/test_identity.csv')
test_identity = test_identity.compute()
test_identity = test_identity.reset_index(drop=True)
print("test_identity finished")
#train_transaction
train_transaction = dd.read_csv('../input/train_transaction.csv')
train_transaction = train_transaction.compute()
train_transaction = train_transaction.reset_index(drop=True)
print("train_transaction finished")
#test_transaction
test_transaction = dd.read_csv('../input/test_transaction.csv')
test_transaction = test_transaction.compute()
test_transaction = test_transaction.reset_index(drop=True)
print("test_transaction finished")
#sample_submission
sample_submission = pd.read_csv('../input/sample_submission.csv')
print(sample_submission.shape)

train_identity finished
test_identity finished
train_transaction finished
test_transaction finished
(506691, 2)
CPU times: user 1min 10s, sys: 11.3 s, total: 1min 21s
Wall time: 28.1 s


In [7]:
print(train_identity.shape,test_identity.shape,train_transaction.shape,test_transaction.shape)

(144233, 41) (141907, 41) (590540, 394) (506691, 393)


In [8]:
%%time
train = pd.merge(train_transaction, train_identity, on='TransactionID', how='left')
test = pd.merge(test_transaction, test_identity, on='TransactionID', how='left')

CPU times: user 7.93 s, sys: 7.01 s, total: 14.9 s
Wall time: 14.9 s


In [9]:
#sample_submission
sample_submission = pd.read_csv('../input/sample_submission.csv')
print(train.shape,test.shape,sample_submission.shape)

(590540, 434) (506691, 433) (506691, 2)


In [10]:
train.head()

,TransactionID,isFraud,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,card6,addr1,addr2,dist1,dist2,P_emaildomain,R_emaildomain,C1,C2,C3,C4,C5,C6,C7,C8,C9,C10,C11,C12,C13,C14,D1,D2,D3,D4,D5,D6,D7,D8,D9,D10,D11,D12,D13,D14,D15,M1,M2,M3,M4,M5,M6,M7,M8,M9,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21,V22,V23,V24,V25,V26,V27,V28,V29,V30,V31,V32,V33,V34,V35,V36,V37,V38,V39,V40,V41,V42,V43,V44,V45,V46,V47,V48,V49,V50,V51,V52,V53,V54,V55,V56,V57,V58,V59,V60,V61,V62,V63,V64,V65,V66,V67,V68,V69,V70,V71,V72,V73,V74,V75,V76,V77,V78,V79,V80,V81,V82,V83,V84,V85,V86,V87,V88,V89,V90,V91,V92,V93,V94,V95,V96,V97,V98,V99,V100,V101,V102,V103,V104,V105,V106,V107,V108,V109,V110,V111,V112,V113,V114,V115,V116,V117,V118,V119,V120,V121,V122,V123,V124,V125,V126,V127,V128,V129,V130,V131,V132,V133,V134,V135,V136,V137,V138,V139,V140,V141,V142,V143,V144,V145,V146,V147,V148,V149,V150,V151,V152,V153,V154,V155,V156,V157,V158,V159,V160,V161,V162,V163,V164,V165,V166,V167,V168,V169,V170,V171,V172,V173,V174,V175,V176,V177,V178,V179,V180,V181,V182,V183,V184,V185,V186,V187,V188,V189,V190,V191,V192,V193,V194,V195,V196,V197,V198,V199,V200,V201,V202,V203,V204,V205,V206,V207,V208,V209,V210,V211,V212,V213,V214,V215,V216,V217,V218,V219,V220,V221,V222,V223,V224,V225,V226,V227,V228,V229,V230,V231,V232,V233,V234,V235,V236,V237,V238,V239,V240,V241,V242,V243,V244,V245,V246,V247,V248,V249,V250,V251,V252,V253,V254,V255,V256,V257,V258,V259,V260,V261,V262,V263,V264,V265,V266,V267,V268,V269,V270,V271,V272,V273,V274,V275,V276,V277,V278,V279,V280,V281,V282,V283,V284,V285,V286,V287,V288,V289,V290,V291,V292,V293,V294,V295,V296,V297,V298,V299,V300,V301,V302,V303,V304,V305,V306,V307,V308,V309,V310,V311,V312,V313,V314,V315,V316,V317,V318,V319,V320,V321,V322,V323,V324,V325,V326,V327,V328,V329,V330,V331,V332,V333,V334,V335,V336,V337,V338,V339,id_01,id_02,id_03,id_04,id_05,id_06,id_07,id_08,id_09,id_10,id_11,id_12,id_13,id_14,id_15,id_16,id_17,id_18,id_19,id_20,id_21,id_22,id_23,id_24,id_25,id_26,id_27,id_28,id_29,id_30,id_31,id_32,id_33,id_34,id_35,id_36,id_37,id_38,DeviceType,DeviceInfo
0,2987000,0,86400,68.5,W,13926,NaN,150.0,discover,142.0,credit,315.0,87.0,19.0,NaN,NaN,NaN,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,2.0,0.0,1.0,1.0,14.0,NaN,13.0,NaN,NaN,NaN,NaN,NaN,NaN,13.0,13.0,NaN,NaN,NaN,0.0,T,T,T,M2,F,T,NaN,NaN,NaN,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,117.0,0.0,0.0,0.0,0.0,0.0,117.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,117.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,117.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2987001,0,86401,29.0,W,2755,404.0,150.0,mastercard,102.0,credit,325.0,87.0,NaN,NaN,gmail.com,NaN,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0

In [11]:
C_columns = [x for x in train.columns if x[0] =='C']
D_columns = [x for x in train.columns if x[0] =='D'][:-2]
M_columns = [x for x in train.columns if x[0] =='M']
V_columns = [x for x in train.columns if x[0] =='V']
print(C_columns)
print("="*100)
print(D_columns)
print("="*100)
print(M_columns)
print("="*100)
print(V_columns)
print("="*100)

['C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'C7', 'C8', 'C9', 'C10', 'C11', 'C12', 'C13', 'C14']
['D1', 'D2', 'D3', 'D4', 'D5', 'D6', 'D7', 'D8', 'D9', 'D10', 'D11', 'D12', 'D13', 'D14', 'D15']
['M1', 'M2', 'M3', 'M4', 'M5', 'M6', 'M7', 'M8', 'M9']
['V1', 'V2', 'V3', 'V4', 'V5', 'V6', 'V7', 'V8', 'V9', 'V10', 'V11', 'V12', 'V13', 'V14', 'V15', 'V16', 'V17', 'V18', 'V19', 'V20', 'V21', 'V22', 'V23', 'V24', 'V25', 'V26', 'V27', 'V28', 'V29', 'V30', 'V31', 'V32', 'V33', 'V34', 'V35', 'V36', 'V37', 'V38', 'V39', 'V40', 'V41', 'V42', 'V43', 'V44', 'V45', 'V46', 'V47', 'V48', 'V49', 'V50', 'V51', 'V52', 'V53', 'V54', 'V55', 'V56', 'V57', 'V58', 'V59', 'V60', 'V61', 'V62', 'V63', 'V64', 'V65', 'V66', 'V67', 'V68', 'V69', 'V70', 'V71', 'V72', 'V73', 'V74', 'V75', 'V76', 'V77', 'V78', 'V79', 'V80', 'V81', 'V82', 'V83', 'V84', 'V85', 'V86', 'V87', 'V88', 'V89', 'V90', 'V91', 'V92', 'V93', 'V94', 'V95', 'V96', 'V97', 'V98', 'V99', 'V100', 'V101', 'V102', 'V103', 'V104', 'V105', 'V106', 'V107', 'V108', '

In [12]:
train['uid'] = train['card1'].astype(str)+'_'+train['card2'].astype(str)
test['uid'] = test['card1'].astype(str)+'_'+test['card2'].astype(str)

train['uid1'] = train['uid'].astype(str)+'_'+train['card3'].astype(str)
test['uid1'] = test['uid'].astype(str)+'_'+test['card3'].astype(str)


train['uid2'] = train['uid'].astype(str)+'_'+train['card3'].astype(str)+'_'+train['card5'].astype(str)
test['uid2'] = test['uid'].astype(str)+'_'+test['card3'].astype(str)+'_'+test['card5'].astype(str)

train['uid3'] = train['uid2'].astype(str)+'_'+train['addr1'].astype(str)+'_'+train['addr2'].astype(str)
test['uid3'] = test['uid2'].astype(str)+'_'+test['addr1'].astype(str)+'_'+test['addr2'].astype(str)

train['uid4'] = train['card4'].astype(str)+'_'+train['card6'].astype(str)
test['uid4'] = test['card4'].astype(str)+'_'+test['card6'].astype(str)

In [13]:
train['TransactionAmt_check'] = np.where(train['TransactionAmt'].isin(test['TransactionAmt']), 1, 0)
test['TransactionAmt_check']  = np.where(test['TransactionAmt'].isin(train['TransactionAmt']), 1, 0)
print("1")
train['C_na'] = abs(train[C_columns].isna().sum(axis=1).astype(np.int8))
test['C_na'] = abs(test[C_columns].isna().sum(axis=1).astype(np.int8))
print("2")
train['D_na'] = abs(train[D_columns].isna().sum(axis=1).astype(np.int8))
test['D_na'] = abs(test[D_columns].isna().sum(axis=1).astype(np.int8))
print("3")
train['M_na'] = abs(train[M_columns].isna().sum(axis=1).astype(np.int8))
test['M_na'] = abs(test[M_columns].isna().sum(axis=1).astype(np.int8))
print("4")
train['V_na'] = abs(train[V_columns].isna().sum(axis=1).astype(np.int8))
test['V_na'] = abs(test[V_columns].isna().sum(axis=1).astype(np.int8))
print("5")

1
2
3
4
5


In [14]:
for i in tqdm(M_columns):
    train[i] = train[i].replace(to_replace = {'F':0,'T':1,'M0':0,'M1':1,'M2':2})
    test[i] = test[i].replace(to_replace = {'F':0,'T':1,'M0':0,'M1':1,'M2':2})

100%|██████████| 9/9 [00:05<00:00,  1.64it/s]


In [15]:
train['C_nanmean'] = train[C_columns].apply(np.nanmean,axis=1)
test['C_nanmean'] = test[C_columns].apply(np.nanmean,axis=1)
print("1")
train['D_nanmean'] = train[D_columns].apply(np.nanmean,axis=1)
test['D_nanmean'] = test[D_columns].apply(np.nanmean,axis=1)
print("2")
train['M_nanmean'] = train[M_columns].apply(np.nanmean,axis=1)
test['M_nanmean'] = test[M_columns].apply(np.nanmean,axis=1)
print("3")
train['V_nanmean'] = train[V_columns].apply(np.nanmean,axis=1)
test['V_nanmean'] = test[V_columns].apply(np.nanmean,axis=1)
print("4")

/usr/local/lib/python3.6/dist-packages/pandas/core/apply.py:242: RuntimeWarning: Mean of empty slice
  labels=labels)


1
2
3
4


In [16]:
train['C_nansum'] = train[C_columns].apply(np.nansum,axis=1)
test['C_nansum'] = test[C_columns].apply(np.nansum,axis=1)
print("1")
train['D_nansum'] = train[D_columns].apply(np.nansum,axis=1)
test['D_nansum'] = test[D_columns].apply(np.nansum,axis=1)
print("2")
train['M_nansum'] = train[M_columns].apply(np.nansum,axis=1)
test['M_nansum'] = test[M_columns].apply(np.nansum,axis=1)
print("3")
train['V_nansum'] = train[V_columns].apply(np.nansum,axis=1)
test['V_nansum'] = test[V_columns].apply(np.nansum,axis=1)
print("4")

1
2
3
4


In [17]:
train['C_nanmedian'] = train[C_columns].apply(np.nanmedian,axis=1)
test['C_nanmedian'] = test[C_columns].apply(np.nanmedian,axis=1)
print("1")
train['D_nanmedian'] = train[D_columns].apply(np.nanmedian,axis=1)
test['D_nanmedian'] = test[D_columns].apply(np.nanmedian,axis=1)
print("2")
train['M_nanmedian'] = train[M_columns].apply(np.nanmedian,axis=1)
test['M_nanmedian'] = test[M_columns].apply(np.nanmedian,axis=1)
print("3")
train['V_nanmedian'] = train[V_columns].apply(np.nanmedian,axis=1)
test['V_nanmedian'] = test[V_columns].apply(np.nanmedian,axis=1)
print("4")

/home/muhammed/.local/lib/python3.6/site-packages/numpy/lib/function_base.py:3400: RuntimeWarning: All-NaN slice encountered
  r = func(a, **kwargs)


1
2
3
4


In [18]:
train['C_nanstd'] = train[C_columns].apply(np.nanstd,axis=1)
test['C_nanstd'] = test[C_columns].apply(np.nanstd,axis=1)
print("1")
train['D_nanstd'] = train[D_columns].apply(np.nanstd,axis=1)
test['D_nanstd'] = test[D_columns].apply(np.nanstd,axis=1)
print("2")
train['M_nanstd'] = train[M_columns].apply(np.nanstd,axis=1)
test['M_nanstd'] = test[M_columns].apply(np.nanstd,axis=1)
print("3")
train['V_nanstd'] = train[V_columns].apply(np.nanstd,axis=1)
test['V_nanstd'] = test[V_columns].apply(np.nanstd,axis=1)
print("4")

/home/muhammed/.local/lib/python3.6/site-packages/numpy/lib/nanfunctions.py:1628: RuntimeWarning: Degrees of freedom <= 0 for slice.
  keepdims=keepdims)


1
2
3
4


In [19]:
train['TransactionAmt_to_mean_card1'] = train['TransactionAmt'] / train.groupby(['card1'])['TransactionAmt'].transform('mean')
train['TransactionAmt_to_mean_card4'] = train['TransactionAmt'] / train.groupby(['card4'])['TransactionAmt'].transform('mean')
train['TransactionAmt_to_std_card1'] = train['TransactionAmt'] / train.groupby(['card1'])['TransactionAmt'].transform('std')
train['TransactionAmt_to_std_card4'] = train['TransactionAmt'] / train.groupby(['card4'])['TransactionAmt'].transform('std')

test['TransactionAmt_to_mean_card1'] = test['TransactionAmt'] / test.groupby(['card1'])['TransactionAmt'].transform('mean')
test['TransactionAmt_to_mean_card4'] = test['TransactionAmt'] / test.groupby(['card4'])['TransactionAmt'].transform('mean')
test['TransactionAmt_to_std_card1'] = test['TransactionAmt'] / test.groupby(['card1'])['TransactionAmt'].transform('std')
test['TransactionAmt_to_std_card4'] = test['TransactionAmt'] / test.groupby(['card4'])['TransactionAmt'].transform('std')

train['id_02_to_mean_card1'] = train['id_02'] / train.groupby(['card1'])['id_02'].transform('mean')
train['id_02_to_mean_card4'] = train['id_02'] / train.groupby(['card4'])['id_02'].transform('mean')
train['id_02_to_std_card1'] = train['id_02'] / train.groupby(['card1'])['id_02'].transform('std')
train['id_02_to_std_card4'] = train['id_02'] / train.groupby(['card4'])['id_02'].transform('std')

test['id_02_to_mean_card1'] = test['id_02'] / test.groupby(['card1'])['id_02'].transform('mean')
test['id_02_to_mean_card4'] = test['id_02'] / test.groupby(['card4'])['id_02'].transform('mean')
test['id_02_to_std_card1'] = test['id_02'] / test.groupby(['card1'])['id_02'].transform('std')
test['id_02_to_std_card4'] = test['id_02'] / test.groupby(['card4'])['id_02'].transform('std')

train['D15_to_mean_card1'] = train['D15'] / train.groupby(['card1'])['D15'].transform('mean')
train['D15_to_mean_card4'] = train['D15'] / train.groupby(['card4'])['D15'].transform('mean')
train['D15_to_std_card1'] = train['D15'] / train.groupby(['card1'])['D15'].transform('std')
train['D15_to_std_card4'] = train['D15'] / train.groupby(['card4'])['D15'].transform('std')

test['D15_to_mean_card1'] = test['D15'] / test.groupby(['card1'])['D15'].transform('mean')
test['D15_to_mean_card4'] = test['D15'] / test.groupby(['card4'])['D15'].transform('mean')
test['D15_to_std_card1'] = test['D15'] / test.groupby(['card1'])['D15'].transform('std')
test['D15_to_std_card4'] = test['D15'] / test.groupby(['card4'])['D15'].transform('std')

train['D15_to_mean_addr1'] = train['D15'] / train.groupby(['addr1'])['D15'].transform('mean')
train['D15_to_mean_card4'] = train['D15'] / train.groupby(['card4'])['D15'].transform('mean')
train['D15_to_std_addr1'] = train['D15'] / train.groupby(['addr1'])['D15'].transform('std')
train['D15_to_std_card4'] = train['D15'] / train.groupby(['card4'])['D15'].transform('std')

test['D15_to_mean_addr1'] = test['D15'] / test.groupby(['addr1'])['D15'].transform('mean')
test['D15_to_mean_card4'] = test['D15'] / test.groupby(['card4'])['D15'].transform('mean')
test['D15_to_std_addr1'] = test['D15'] / test.groupby(['addr1'])['D15'].transform('std')
test['D15_to_std_card4'] = test['D15'] / test.groupby(['card4'])['D15'].transform('std')

In [20]:
train['TransactionAmt_decimal'] = train['TransactionAmt'].apply(lambda x: x % 1)
test['TransactionAmt_decimal'] = test['TransactionAmt'].apply(lambda x: x % 1)

In [21]:
train['TransactionAmt_decimal'] = train['TransactionAmt_decimal'].apply(lambda x: np.round(x,2))
test['TransactionAmt_decimal'] = test['TransactionAmt_decimal'].apply(lambda x: np.round(x,2))

In [22]:
train['TransactionAmt_decimal_0.0'] = train['TransactionAmt_decimal'].apply(lambda x: 1 if x == 0 else 0)
train['TransactionAmt_decimal_0.5'] = train['TransactionAmt_decimal'].apply(lambda x: 1 if x == 0.5 else 0)
train['TransactionAmt_decimal_1.0'] = train['TransactionAmt_decimal'].apply(lambda x: 1 if x == 1.0 else 0)

test['TransactionAmt_decimal_0.0'] = test['TransactionAmt_decimal'].apply(lambda x: 1 if x == 0 else 0)
test['TransactionAmt_decimal_0.5'] = test['TransactionAmt_decimal'].apply(lambda x: 1 if x == 0.5 else 0)
test['TransactionAmt_decimal_1.0'] = test['TransactionAmt_decimal'].apply(lambda x: 1 if x == 1.0 else 0)


In [23]:
print(train.shape,test.shape)

(590540, 478) (506691, 477)


In [24]:
START_DATE = datetime.datetime.strptime('2017-11-30', '%Y-%m-%d')

In [25]:
for df in tqdm([train, test]):
    # Temporary
    df['DT'] = df['TransactionDT'].apply(lambda x: (START_DATE + datetime.timedelta(seconds = x)))
    df['month'] = (df['DT'].dt.year-2017)*12 + df['DT'].dt.month
    df['weekofyear'] = (df['DT'].dt.year-2017)*52 + df['DT'].dt.weekofyear
    df['dayofyear'] = (df['DT'].dt.year-2017)*365 + df['DT'].dt.dayofyear
    
    df['hour'] = df['DT'].dt.hour
    df['dayofweek'] = df['DT'].dt.dayofweek
    df['day'] = df['DT'].dt.day

100%|██████████| 2/2 [00:01<00:00,  1.43it/s]


In [26]:
a_ = train['dayofyear'].min()
train['relative_hour'] = (train['dayofyear'] - a_)*24 + train['hour']
test['relative_hour'] = (test['dayofyear'] - a_)*24 + test['hour']

In [27]:
train['relative_hour_count'] = train['relative_hour'].map(train['relative_hour'].value_counts().to_dict())
test['relative_hour_count'] = test['relative_hour'].map(test['relative_hour'].value_counts().to_dict())

In [28]:
temp = pd.concat([train['hour'],test['hour']],axis=0)
temp = temp.value_counts().to_dict()
train['hour_count'] = train['hour'].map(train['hour'].value_counts().to_dict())
train['hour_count_merged'] = train['hour'].map(temp)
test['hour_count'] = test['hour'].map(test['hour'].value_counts().to_dict())
test['hour_count_merged'] = test['hour'].map(temp)

In [29]:
train['card1_count_full'] = train['card1'].map(pd.concat([train['card1'], test['card1']], ignore_index=True).value_counts(dropna=False))
test['card1_count_full'] = test['card1'].map(pd.concat([train['card1'], test['card1']], ignore_index=True).value_counts(dropna=False))
print("1")
train['card2_count_full'] = train['card2'].map(pd.concat([train['card2'], test['card2']], ignore_index=True).value_counts(dropna=False))
test['card2_count_full'] = test['card2'].map(pd.concat([train['card2'], test['card2']], ignore_index=True).value_counts(dropna=False))
print("2")
train['card3_count_full'] = train['card3'].map(pd.concat([train['card3'], test['card3']], ignore_index=True).value_counts(dropna=False))
test['card3_count_full'] = test['card3'].map(pd.concat([train['card3'], test['card3']], ignore_index=True).value_counts(dropna=False))
print("3")
train['card4_count_full'] = train['card4'].map(pd.concat([train['card4'], test['card4']], ignore_index=True).value_counts(dropna=False))
test['card4_count_full'] = test['card4'].map(pd.concat([train['card4'], test['card4']], ignore_index=True).value_counts(dropna=False))
print("4")
train['card5_count_full'] = train['card5'].map(pd.concat([train['card5'], test['card5']], ignore_index=True).value_counts(dropna=False))
test['card5_count_full'] = test['card5'].map(pd.concat([train['card5'], test['card5']], ignore_index=True).value_counts(dropna=False))
print("5")
train['card6_count_full'] = train['card6'].map(pd.concat([train['card6'], test['card6']], ignore_index=True).value_counts(dropna=False))
test['card6_count_full'] = test['card6'].map(pd.concat([train['card6'], test['card6']], ignore_index=True).value_counts(dropna=False))
print("6")

1
2
3
4
5
6


In [30]:
del train['DT'],train['month'],train['weekofyear'],train['dayofyear'],train['hour'],train['dayofweek'],train['day'],train['TransactionAmt_decimal']
del test['DT'],test['month'],test['weekofyear'],test['dayofyear'],test['hour'],test['dayofweek'],test['day'],test['TransactionAmt_decimal']

In [31]:
for feature in ['uid1','uid4']:
    le = LabelEncoder()
    le.fit(list(train[feature].astype(str).values) + list(test[feature].astype(str).values))
    train[feature] = le.transform(list(train[feature].astype(str).values))
    test[feature] = le.transform(list(test[feature].astype(str).values))

In [32]:
def reduce_mem_usage(df):
    """ iterate through all the columns of a dataframe and modify the data type
        to reduce memory usage.        
    """
    start_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage of dataframe is {:.2f} MB'.format(start_mem))
    
    for col in df.columns:
        col_type = df[col].dtype
        
        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
        else:
            df[col] = df[col].astype('category')

    end_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))
    
    return df

In [33]:
train['TransactionID'].head()

0    2987000
1    2987001
2    2987002
3    2987003
4    2987004
Name: TransactionID, dtype: int64

In [34]:
[x for x in train.columns if x not in test.columns]

['isFraud']

In [35]:
train = train[ ['TransactionID'] + list(train.columns[-50:])]
test = test[ ['TransactionID'] + list(train.columns[-50:])]
print(train.shape,test.shape)

(590540, 51) (506691, 51)


In [36]:
train = reduce_mem_usage(train)
test = reduce_mem_usage(test)
train.to_csv('../input/train_30eylul.csv',index=False)
test.to_csv('../input/test_30eylul.csv',index=False)

Memory usage of dataframe is 218.51 MB
Memory usage after optimization is: 80.42 MB
Decreased by 63.2%
Memory usage of dataframe is 187.49 MB


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-vers

Memory usage after optimization is: 74.03 MB
Decreased by 60.5%


# Training

In [ ]:
train.drop(['TransactionID'],axis=1,inplace=True)
test.drop(['TransactionID'],axis=1,inplace=True)
gc.collect()

In [ ]:
y = train['isFraud']
del train['isFraud']

In [ ]:
########################### Model params
lgb_params = {'num_leaves': 546,
          'min_child_weight': 0.03454472573214212,
          'feature_fraction': 0.1797454081646243,
          'bagging_fraction': 0.2181193142567742,
          'min_data_in_leaf': 106,
          'objective': 'binary',
          'max_depth': -1,
          'learning_rate': 0.01,
          "boosting_type": "gbdt",
          "bagging_seed": 11,
          "metric": 'auc',
          "verbosity": -1,
          'reg_alpha': 0.3299927210061127,
          'reg_lambda': 0.3885237330340494,
          'random_state': 42,
}

In [ ]:
train.drop(['C13','C14','D3'],axis=1,inplace=True)
test.drop(['C13','C14','D3'],axis=1,inplace=True)

In [ ]:
N = 5
kf = KFold(n_splits=N)

importance = pd.DataFrame(np.zeros((train.shape[1], N)), columns=['Fold_{}'.format(i) for i in range(1, N + 1)], index=train.columns)
scores = []
y_pred = np.zeros(test.shape[0])
oof = np.zeros(train.shape[0])

for fold, (trn_idx, val_idx) in enumerate(kf.split(train, y), 1):
    print('Fold {}'.format(fold))
          
    trn_data = lgb.Dataset(train.iloc[trn_idx, :].values, label=y.iloc[trn_idx].values)
    val_data = lgb.Dataset(train.iloc[val_idx, :].values, label=y.iloc[val_idx].values)   
    
    #params, dtrain, 1600, valid_sets = [dtrain, dvalid], verbose_eval=200, early_stopping_rounds=500
    clf = lgb.train(lgb_params, trn_data ,2000 ,valid_sets=[trn_data, val_data], verbose_eval=200,early_stopping_rounds=500)

    predictions = clf.predict(train.iloc[val_idx, :].values) 
    importance.iloc[:, fold - 1] = clf.feature_importance()
    oof[val_idx] = predictions

    score = roc_auc_score(y.iloc[val_idx].values, predictions)
    scores.append(score)
    print('Fold {} ROC AUC Score {}\n'.format(fold, score))

    y_pred += clf.predict(test) / N
    
    del trn_data, val_data, predictions
    gc.collect()
    
print('Average ROC AUC Score {} [STD:{}]'.format(np.mean(scores), np.std(scores)))

In [ ]:
print(roc_auc_score(y,oof))

In [ ]:
importance['avg'] = importance.mean(axis=1)

In [ ]:
importance = importance.sort_values(by='avg',ascending=False)

In [ ]:
importance = importance.reset_index(drop=False)

In [ ]:
importance

In [ ]:
importance.to_csv('../importances/importances24.csv',index=False)

In [ ]:
sample_submission.head()

In [ ]:
sample_submission['isFraud'] = y_pred

In [ ]:
sample_submission.head()

In [ ]:
sample_submission.to_csv('../predictions/pred24.csv',index=False)

In [ ]:
!kaggle competitions submit -c ieee-fraud-detection -f ../predictions/pred24.csv -m "pred24"